In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import pandas as pd

In [6]:
df = pd.read_csv('traindata.csv')
dropped = ['processed', 'offensiveness_score']
rename = {'txt': 'comment', 'isOffensive': 'label'}
df = df.drop(columns=dropped)
df = df.rename(columns=rename)
df.head()

,comment,label
0,> The difference in average earnings between m...,0
1,"The myth is that the ""gap"" is entirely based o...",0
2,The assertion is that women get paid less for ...,0
3,You said in the OP that's not what they're mea...,0
4,>Men and women are not payed less for the same...,0


In [7]:
comments = df['comment'].tolist()
labels = df['label'].tolist()

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_inputs = tokenizer(comments, padding=True, truncation=True, return_tensors='tf')

In [11]:
labels = tf.convert_to_tensor(labels)

In [12]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [13]:
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

In [26]:
outputs = bert_model(input_ids, attention_mask=attention_mask)
pooled_output = outputs['pooler_output']


ResourceExhaustedError: Exception encountered when calling layer 'embeddings' (type TFBertEmbeddings).

{{function_node __wrapped__ResourceGather_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[165131,512,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:ResourceGather]

Call arguments received by layer 'embeddings' (type TFBertEmbeddings):
  • input_ids=tf.Tensor(shape=(165131, 512), dtype=int32)
  • position_ids=None
  • token_type_ids=tf.Tensor(shape=(165131, 512), dtype=int32)
  • inputs_embeds=None
  • past_key_values_length=0
  • training=False

In [21]:
discriminator = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics=['accuracy'])

In [25]:
for batch_output in batch_generator(input_ids, attention_mask, batch_size):
    discriminator.fit(batch_output, labels, epochs=1)

ValueError: Data cardinality is ambiguous:
  x sizes: 16
  y sizes: 165131
Make sure all arrays contain the same number of samples.